In [19]:
import os
import sys
import django
import pandas as pd
import numpy as np

cwd = os.getcwd()

BASE_DIR = os.path.abspath(os.path.join(cwd, '..', '..'))  

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)
    print(f"Adicionado {BASE_DIR} ao sys.path")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "projetob.settings")

django.setup()

from api.models import *
from asgiref.sync import sync_to_async

In [20]:
async def get_marketdata_df():
    qs = await sync_to_async(list)(MarketData.objects.all().values())
    df = pd.DataFrame(qs)
    return df

# Executa e recebe o DataFrame
df_marketdata = await get_marketdata_df()
print(df_marketdata.head())

      id        date  close   high    low   open  volume symbol
0  17344  1927-12-30  17.66  17.66  17.66  17.66       0  ^GSPC
1  17345  1928-01-03  17.76  17.76  17.76  17.76       0  ^GSPC
2  17346  1928-01-04  17.72  17.72  17.72  17.72       0  ^GSPC
3  17347  1928-01-05  17.55  17.55  17.55  17.55       0  ^GSPC
4  17348  1928-01-06  17.66  17.66  17.66  17.66       0  ^GSPC


In [21]:
df_marketdata.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280087 entries, 0 to 280086
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      280087 non-null  int64 
 1   date    280087 non-null  object
 2   close   280087 non-null  object
 3   high    280087 non-null  object
 4   low     280087 non-null  object
 5   open    280087 non-null  object
 6   volume  280087 non-null  int64 
 7   symbol  280087 non-null  object
dtypes: int64(2), object(6)
memory usage: 17.1+ MB


In [22]:
print("Symbols no BD:", df_marketdata['symbol'].unique())

Symbols no BD: ['^GSPC' 'M2SL' '^FVX' 'NFCI' 'SPGI' 'DHR' 'AAPL' 'UNH' 'TEDRATE' 'MSFT'
 '^BVSP' 'BRFS' 'AMZN' 'NVDA' 'ITSA4.SA' 'PETR4.SA' 'VALE3.SA' 'WEGE3.SA'
 'ABEV3.SA' 'DX=F' 'ZC=F' 'CL=F' 'GC=F' 'NG=F' 'ZS=F' 'ITUB4.SA' 'BRL=X'
 'GOOGL' 'Selic_Over' 'Selic_Over_Long' 'CDI' 'Swap_DI_5Y' 'CSAN3.SA'
 'GSG' 'BZ=F' 'B3SA3.SA' 'CDS_Brasil_5Y' 'BBDC4.SA' 'BTGIMABFIRF.SA'
 'CDS_USA_5Y' 'TSLA' 'META' 'BTC-USD' 'NTNS11.SA']


In [23]:
async def get_predictiondata_df():
    qs = await sync_to_async(list)(Prediction.objects.all().values())
    return pd.DataFrame(qs)

df_prediction = await get_predictiondata_df()
df_prediction.head()

,id,date,results,symbol,prediction
0,65,2025-10-16,"{'history': {'loss': [0.005554144736379385, 0....",AAPL,"[236.6743927001953, 235.437744140625, 235.1471..."
1,66,2025-10-16,"{'history': {'loss': [0.011958727613091469, 0....",ABEV3.SA,"[11.814314842224121, 11.907608032226562, 11.84..."
2,67,2025-10-16,"{'history': {'loss': [0.008539831265807152, 0....",AMZN,"[218.88583374023438, 220.56967163085938, 218.8..."
3,68,2025-10-16,"{'history': {'loss': [0.00551522197201848, 0.0...",B3SA3.SA,"[12.833183288574219, 12.811125755310059, 12.85..."
4,69,2025-10-16,"{'history': {'loss': [0.008469405584037304, 0....",BBDC4.SA,"[17.071815490722656, 17.06732749938965, 17.035..."


In [28]:
metrics = []
for j, row in df_prediction.iterrows():
    print(f'{row['symbol']}, {row['date']}: {row['results']['metrics']}. {row['results']['selected_features']}')
    metrics.append(row['results']['metrics'])


AAPL, 2025-10-16: {'loss': 0.0016586844576522708, 'mae': 0.032063182443380356}. [['MSFT', 0.9844172373811919], ['DX=F', 0.6697946962078916], ['CSAN3.SA', 0.6597146625370014], ['^FVX', 0.653657570842682]]
ABEV3.SA, 2025-10-16: {'loss': 0.000304333952954039, 'mae': 0.012372687458992004}. [['GSG', -0.7134098212123902], ['BBDC4.SA', 0.6522477825272643], ['DX=F', 0.632422341174613], ['^GSPC', 0.467580294454299]]
AMZN, 2025-10-16: {'loss': 0.0016062386566773057, 'mae': 0.03145165741443634}. [['SPGI', 0.9760020289879526], ['BBDC4.SA', 0.7231559284175962], ['CSAN3.SA', 0.7044271477272491], ['DX=F', 0.6451656779646893]]
B3SA3.SA, 2025-10-16: {'loss': 0.0005087834433652461, 'mae': 0.01698801852762699}. [['^BVSP', 0.9019386876012959], ['CSAN3.SA', 0.8109475942163319], ['TEDRATE', -0.4744264686252334], ['DX=F', 0.521851167823445]]
BBDC4.SA, 2025-10-16: {'loss': 0.0004614948411472142, 'mae': 0.015609415248036385}. [['ITUB4.SA', 0.8228485451791256], ['GSG', -0.609100078420222], ['CSAN3.SA', 0.695619

In [25]:
metrics_df = pd.DataFrame(metrics)
metrics_df.describe()

,loss,mae
count,72.000000,72.000000
mean,0.002026,0.026012
std,0.005138,0.024275
min,0.000013,0.003209
25%,0.000402,0.014249
50%,0.000633,0.018658
75%,0.001408,0.027992
max,0.036545,0.163892


In [26]:
async def get_portfolios_df():
    qs = await sync_to_async(list)(Portfolio.objects.all().values())
    return pd.DataFrame(qs)

df_portfolios = await get_portfolios_df()
df_portfolios.head()

,id,user_id,name,description,created_at,assets,initial_distribution,current_distribution,initial_balance,current_balance
0,8,2,123213,isso.,2025-10-16 22:59:06.836892+00:00,"[{'symbol': 'AAPL', 'price': 150.5, 'quantity'...","{'AAPL': 0.682153, 'GOOGL': 0.317847}","{'AAPL': 0.6652, 'GOOGL': 0.3348}",2206.25,3748.55
1,7,1,Carteira Exótica corr,Carteira para fins do coisa ruim. corr,2025-10-15 20:45:21.482036+00:00,"[{'symbol': 'SPGI', 'price': 150, 'quantity': ...","{'SPGI': 0.252525, 'AAPL': 0.656566, 'DHR': 0....","{'SPGI': 0.4167, 'AAPL': 0.5303, 'DHR': 0.053}",11880.00,23359.20
2,6,1,Carteira Exótica,Carteira para fins do coisa ruim.,2025-10-15 20:39:17.288123+00:00,"[{'symbol': 'SPGI', 'price': 150, 'quantity': ...","{'SPGI': 0.252525, 'AAPL': 0.747475}","{'SPGI': 0.8675, 'AAPL': 0.1325}",11880.00,11220.42
3,5,1,Meu Portfolio Teste2EWE,Portfolio para testar a lógica2WEWE,2025-10-14 21:36:10.710832+00:00,"[{'symbol': 'PETR4.SA', 'price': 25.5, 'quanti...","{'PETR4.SA': 0.459459, 'VALE3.SA': 0.540541}","{'PETR4.SA': 0.503, 'VALE3.SA': 0.497000000000...",5550.00,6010.50
4,4,1,Meu Portfolio Teste2,Portfolio para testar a lógica2,2025-10-14 21:35:27.625489+00:00,"[{'symbol': 'PETR4.SA', 'price': 25.5, 'quanti...","{'PETR4.SA': 0.459459, 'VALE5.SA': 0.540541}",{'PETR4.SA': 1.0},5550.00,3023.00


In [27]:
for i in df_portfolios.assets:
    print(i)

[{'symbol': 'AAPL', 'price': 150.5, 'quantity': 10}, {'symbol': 'GOOGL', 'price': 140.25, 'quantity': 5}]
[{'symbol': 'SPGI', 'price': 150, 'quantity': 20}, {'symbol': 'AAPL', 'price': 156, 'quantity': 50}, {'symbol': 'DHR', 'price': 180, 'quantity': 6}]
[{'symbol': 'SPGI', 'price': 150, 'quantity': 20}, {'symbol': 'AAPL', 'price': 156, 'quantity': 50}, {'symbol': 'AAPL', 'price': 180, 'quantity': 6}]
[{'symbol': 'PETR4.SA', 'price': 25.5, 'quantity': 100}, {'symbol': 'VALE3.SA', 'price': 60.0, 'quantity': 50}]
[{'symbol': 'PETR4.SA', 'price': 25.5, 'quantity': 100}, {'symbol': 'VALE5.SA', 'price': 60.0, 'quantity': 50}]
[{'symbol': 'PETR4', 'price': 25.5, 'quantity': 100}, {'symbol': 'VALE5', 'price': 60.0, 'quantity': 50}]
[{'symbol': 'GSG', 'price': 18.5, 'quantity': 100}, {'symbol': 'ITSA4.SA', 'price': 9.75, 'quantity': 200}, {'symbol': 'ITUB4.SA', 'price': 26.3, 'quantity': 150}, {'symbol': 'M2SL', 'price': 8.2, 'quantity': 120}, {'symbol': 'META', 'price': 298.5, 'quantity': 10}